<a href="https://colab.research.google.com/github/dahlia52/Advanced-Statistical-Data-Analysis/blob/main/multilayer_logistic_regression_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
num_sample = 250

# 데이터 생성
data0 = np.random.randn(num_sample, 2) + (2,2)
data1 = np.random.randn(num_sample, 2) + (-2,-2)
data2 = np.random.randn(num_sample, 2) + (2,-2)
data3 = np.random.randn(num_sample, 2) + (-2,2)

data0 = np.hstack([data0, np.zeros((num_sample, 1), dtype = float)])
data1 = np.hstack([data1, np.zeros((num_sample, 1), dtype = float)])
data2 = np.hstack([data2, np.ones((num_sample, 1), dtype = float)])
data3 = np.hstack([data3, np.ones((num_sample, 1), dtype = float)])

data = np.vstack([data0, data1, data2, data3])

data = torch.tensor(data, dtype = torch.float32) # data를 tensor로 변환
print(data.shape)

torch.Size([1000, 3])


In [ ]:
class LogisticRegression(nn.Module):
  def __init__(self):
    super().__init__()

    # input.dims = 2 -> output.dims = 1
    self.linear1 = nn.Linear(2,3)
    self.linear2 = nn.Linear(3,3)
    self.linear3 = nn.Linear(3,1)

  def forward(self, x):
    # logit = self.linear(x)
    # output = F.sigmoid(logit)
    hidden1 = F.leaky_relu(self.linear1(x))
    hidden2 = F.tanh(self.linear2(hidden1))
    #output = F.sigmoid(self.linear3(hidden2)) # 0~1 -> log ... + delta
    output = self.linear3(hidden2) # logit = inverse of sigmoid (softmax)
    return output

In [ ]:
model = LogisticRegression()
#loss = nn.BCELoss() # need sigmoid activation
loss = nn.BCEWithLogitsLoss() # do not need sigmoid, only need (pre-sigmoid) logit value
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)

In [ ]:
num_epoch = 200
loss_list = list()

In [ ]:
for i in range(num_epoch):
  perm = np.random.permutation(len(data))
  total_loss = 0
  count = 0

  for j in range(len(data)):
    x = data[perm[j]][:-1]
    y = data[perm[j]][-1] # (1,1)

    y = y.reshape([1]) # (1,)
    #out = model.forward(x) # P(y=1lx)
    logit = model.forward(x)
    prob = F.sigmoid(logit)

    #cost = loss(out, y)
    cost = loss(logit, y)

    optimizer.zero_grad() # gradient를 0으로 초기화
    cost.backward() # cost를 미분
    optimizer.step() # SGD로 gradient update

    total_loss += cost.item()

    pred = torch.round(prob).item()
    count += (pred == y.item())

  acc = count / len(data)
  loss_list.append(total_loss)

  if i % 10 == 0:
    print("Epoch %d: Loss %.3f / Acc %.3f" %(i, total_loss, acc))

Epoch 0: Loss 591.218 / Acc 0.695
Epoch 10: Loss 441.412 / Acc 0.665
Epoch 20: Loss 431.252 / Acc 0.623
Epoch 30: Loss 430.391 / Acc 0.672
Epoch 40: Loss 429.751 / Acc 0.668
Epoch 50: Loss 424.906 / Acc 0.680
Epoch 60: Loss 431.054 / Acc 0.712
Epoch 70: Loss 429.694 / Acc 0.671
Epoch 80: Loss 426.548 / Acc 0.675
Epoch 90: Loss 426.553 / Acc 0.691
Epoch 100: Loss 429.623 / Acc 0.707
Epoch 110: Loss 423.722 / Acc 0.716
Epoch 120: Loss 421.024 / Acc 0.673
Epoch 130: Loss 423.879 / Acc 0.702
Epoch 140: Loss 422.192 / Acc 0.718
Epoch 150: Loss 421.226 / Acc 0.733
Epoch 160: Loss 424.510 / Acc 0.722
Epoch 170: Loss 422.459 / Acc 0.698
Epoch 180: Loss 423.448 / Acc 0.740
Epoch 190: Loss 244.321 / Acc 0.925
